# preprocess essential climate variables

In [ ]:
import os
import numpy as np
import pandas as pd
import netCDF4 as nc
import xarray as xr
import geopandas as gpd

In [ ]:
def select_bounding_box_region(path, years, ecv, lon_min, lon_max, lat_min, lat_max):
    
    data_final = pd.DataFrame(columns=['lat', 'lon', 'time', ecv])
    
    for year in years:
        files = os.listdir(path+year)
        
        for file in files:
            data = xr.open_dataset(path+year+'/'+file)
            data = data.to_dataframe().reset_index()
            data_filtered = data[(data.lon <= lon_max) & (data.lon >= lon_min) & (data.lat >= lat_min) & (data.lat <= lat_max)].dropna(subset=[ecv])[['lat', 'lon', 'time', ecv]]
            data_final = data_final.append(data_filtered)
            
    data_final = data_final.reset_index(drop=True)
    
    return data_final

In [ ]:
years = ['2010',
         '2011',
         '2012',
         '2013',
         '2014',
         '2015']

In [ ]:
lon_min, lon_max, lat_min, lat_max = 60, 100, 0, 40

## sea surface salinity

In [ ]:
!bash download_sea_surface_salinity_data.sh

In [ ]:
path = '../data/sea_surface_salinity/'

In [ ]:
data = nc.Dataset(path+years[0]+'/'+sorted(os.listdir(path+years[0]))[0])

In [ ]:
for var in data.variables.values():
    print(var)

In [ ]:
%%time

sss = select_bounding_box_region(path, years, 'sss', lon_min, lon_max, lat_min, lat_max)

In [ ]:
sss.shape

In [ ]:
sss.info()

In [ ]:
sss.head()

In [ ]:
sss['year'] = sss.time.apply(lambda x: x.year)

In [ ]:
sss['month'] = sss.time.apply(lambda x: x.month)

In [ ]:
sss['month'] = sss.month.astype(str).str.pad(2, side='left',fillchar='0')

In [ ]:
sss = sss[['lat', 'lon', 'year', 'month', 'sss']].groupby(['lat', 'lon', 'year', 'month']).aggregate('mean').reset_index()

In [ ]:
sss.shape

In [ ]:
sss.info()

In [ ]:
sss.head()

In [ ]:
sss = gpd.GeoDataFrame(sss, geometry=gpd.points_from_xy(sss.lon, sss.lat), crs={'init': 'epsg:4326'})

In [ ]:
sss.drop(['lat', 'lon'], axis=1, inplace=True)

In [ ]:
sss.info()

In [ ]:
sss.to_file(path+'monthly_sss_2010_2015.shp')

## chlorophyll-a concentration

In [ ]:
!bash download_chlorophyll_a_concentration_data.sh

In [ ]:
path = '../data/chlorophyll_a_concentration/'

In [ ]:
data = nc.Dataset(path+years[0]+'/'+sorted(os.listdir(path+years[0]))[0])

In [ ]:
for var in data.variables.values():
    print(var)

In [ ]:
%%time

chlor_a = select_bounding_box_region(path, years, 'chlor_a', lon_min, lon_max, lat_min, lat_max)

In [ ]:
chlor_a.shape

In [ ]:
chlor_a.info()

In [ ]:
chlor_a.head()

In [ ]:
chlor_a['year'] = chlor_a.time.apply(lambda x: x.year)

In [ ]:
chlor_a['month'] = chlor_a.time.apply(lambda x: x.month)

In [ ]:
chlor_a['month'] = chlor_a.month.astype(str).str.pad(2, side='left',fillchar='0')

In [ ]:
chlor_a['chlor_a_log'] = np.log(chlor_a['chlor_a'])

In [ ]:
chlor_a.drop(['time', 'chlor_a'], axis=1, inplace=True)

In [ ]:
chlor_a.head()

In [ ]:
chlor_a = gpd.GeoDataFrame(chlor_a, geometry=gpd.points_from_xy(chlor_a.lon, chlor_a.lat), crs={'init': 'epsg:4326'})

In [ ]:
chlor_a.drop(['lat', 'lon'], axis=1, inplace=True)

In [ ]:
chlor_a.info()

In [ ]:
chlor_a.to_file(path+'monthly_chlor_a_2010_2015.shp')